# Part One

## Data Engineering with Python & AI:
### Data Loading 

1. Extracting Data from APIs & Handling API Challenges - Work with REST APIs, authentication, rate limits, retries, and pagination to extract data efficiently.
2. Schema Management & Automatic Normalization - Use dtl to infer schemas, flatten nested JSON, extract lists into child tables, and handle schema evolution automatically.
3. Incremental Data Extraction & State Tracking - Load only new or modified records, avoiding unnecessary reprocessing and improving pipeline efficiency.
4. Loading Data Into Various Destinations - Store data in DuckDB or Postgres, BigQuery, Snowflake, or a Data Lake while ensuring efficient schema mapping and performance.
5. Automating and Orchestrating Pipelines - Deploy, schedule, and maintain ingestion workflows with Dagster, Github Actions, and Cron Jobs.
6. Scaling Data Pipelines Efficiently - Handle large-scale data ingestion while optimizing performance, retries, and parallel execution.

After this exercise, you won't just know ingestion - you'll be able to build an API ingestion pipeline that `auto-detects schema changes`, `retries intelligently`, and `scales with demand`. (resilient, scalable, efficient and reliable pipeline)

# Data Ingestion
We'll extract data from a source. It often includes normalizing, cleaning, and adding metadata.

## Extracting Data: 
Data Streaming and Batching
* Batching - Processing data in chunks at scheduled intervals. It's suitable for scheduled tasks and reduces system load.
* Streaming - Processing data continuously as it arrives.It's ideal for real-time data processing and inmmediate insights.

Choosing the right approach depends on factors like `data volume`, `latency requirements` and `system architecture`.


1. Batch processing
Batch processing is best when you can wait for data to accumulate before processing it in large chunks. It is `cost-efficient` and works well for `non-time-sensitive` workloads.
- Common use cases
    * Nightly database updates.
    * Generating daily or weekly reports.

2. Streaming data processing
Streaming is useful when you need to `process data in real-time` or `with minimal delay`. Instead of waiting for a batch, events are processed continuously.
- Common use cases
    * Fraud detection (e.g. analyzing transactions in real-time)
    * IoT devide monitoring (e.g. temperature sensors)
    * Event-driven applications (e.g. user activity tracking)
    * Log and telemetry data ingestion

3. When to use Batch vs Streaming

|Factor      |        Batch processing      |              Streaming processing        |
|------------|------------------------------|------------------------------------------|
|Latency     |   High (minutes, hours)      |   Low (milliseconds, seconds)            |
|Data volume |   Large batches              |   Continuous small events                |
|Use case    |   Reports, ETL, backups      |   Real-time analytics, event-driven apps |
|Complexity  |   Easier to manage           |   Requires event-driven architecture     |
|Cost        |   Lower for periodic runs    |   Higher for always-on processing        |

4. Tools
Many tools support both `batch` and `streaming` data extraction. Some tools are optimized for one approach, while others provide flexibility for both.

`Message queues & Event streaming`
These tools enable real-time data ingestion and processing but can also buffer data for mini-batch processing.

  * `Apache Kafka` - Distributed event streaming platformfor real-time and batch        workloads.
  * `RabbitMQ` - Message broker that supports real-time message passing.
  * `AWS Kinesis` - Cloud-native alternative to Kafka for real-time ingestion.
  * `Google Pub/Sub` - Managed messaging service for real-time and batch workloads.

`ETL & ELT Pipelines`
These tools handle extraction, transformation and loading (ETL) for both batch and streaming pipelines.
  
  * `Apache Spark` - Supports batch processing and structured streaming.
  * `dbt (Data Build Tool)` - Focuses on batch transformations but can be used with streaming inputs.
  * `Flink` - Real-time stream processing but can also handle mini-batch workloads.
  * `NiFi` - A data flow tool for moving and transforming data in real time or batch.
  * `AWS Glue` - Serverless ETL service for batch workloads, with limited streaming support.
  * `Google Cloud Dataflow` - Managed ETL platform supporting both batch and streaming.
  * `dlt` - Automates API extraction, incremental ingestion, and schema evolution for both batch and streaming pipelines. 

### Working with RestAPI

#### APIs as a data source: Batch vs. Streaming approaches

APIs are a major source of data ingestion. Depending on how APIs provide data, they can be used in both `batch` and `streaming` workflows.

1. **APIs for batch extraction**

Some APIs return large datasets at once. This data is often fetched on a schedule or as part of an ETL process.

**Common batch API examples:**

   - **CRM APIs (Salesforce, HubSpot)** - Export customer data daily.
   - **E-commerce APIs (Shopify, Amazon)** - Download product catalogs or sales reports periodically.
   - **Public APIs (Weather, Financial Data)** - Retrieve daily stock market updates.

**How batch API extraction works:**
1. Call an API at **scheduled intervals** (e.g. every hour or day).
2. Retrieve all available data (e.g. last 24 hours of records).
3. Store results in a database, data warehouse, or file storage.

In [8]:
#%pip install requests

In [9]:
import requests
import json

def fetch_batch_data():
    url = "https://api.example.com/daily_reports"
    response = requests.get(url) 
    data = response.json()

    with open("daily_report.json", "w") as file:
        json.dump(data, file)


fetch_batch_data()

ConnectionError: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /daily_reports (Caused by NameResolutionError("HTTPSConnection(host='api.example.com', port=443): Failed to resolve 'api.example.com' ([Errno 11001] getaddrinfo failed)"))

In [10]:
import requests


url = "https://api.github.com/repos/DataTalksClub/data-engineering-zoomCamp/events"

response = requests.get(url)
response.json()

[{'id': '5414379751',
  'type': 'WatchEvent',
  'actor': {'id': 13095925,
   'login': 'piyush-24sharma',
   'display_login': 'piyush-24sharma',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/piyush-24sharma',
   'avatar_url': 'https://avatars.githubusercontent.com/u/13095925?'},
  'repo': {'id': 419661684,
   'name': 'DataTalksClub/data-engineering-zoomcamp',
   'url': 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp'},
  'payload': {'action': 'started'},
  'public': True,
  'created_at': '2025-12-19T15:40:31Z',
  'org': {'id': 72699292,
   'login': 'DataTalksClub',
   'gravatar_id': '',
   'url': 'https://api.github.com/orgs/DataTalksClub',
   'avatar_url': 'https://avatars.githubusercontent.com/u/72699292?'}},
 {'id': '5411726096',
  'type': 'WatchEvent',
  'actor': {'id': 1312845,
   'login': 'qivhou',
   'display_login': 'qivhou',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/qivhou',
   'avatar_url': 'https://avatars.githubuserco

2. **APIs for streaming data extraction**

Some APIs support **event-driven** data extraction, where updates are pushed in real-time. This method is used for systems that require immediate action on new data.

**Common streaming API examples:**

   - **Webhooks (Stripe, Github, Slack)** - Real-time event notifications.
   - **Social Media APIs (Twitter Streaming, Reddit Firehose)** - Continuous data from user interactions.
   - **Financial Market APIs (Binance WebSocket, AlphaVantage Streaming)** - Live stock prices and cryptocurrency trades.

**How streaming API extraction works:**

1. API sends **real-time updates** as data changes.
2. A webhook or WebSocket **listens for events**.
3. Data is **processed immediately** instead of being stored in bulk.

In [ ]:
#%pip install websocket

In [11]:
import websocket

def on_message(ws, message):
    print("Recieved event:", message)

ws = websocket.WebSocketApp("wss://api.example.com/stream", on_message=on_message) # We do not request we subscribe.
ws.run_forever()

AttributeError: module 'websocket' has no attribute 'WebSocketApp'

As an engineer, you will need to build pipelines that "just work".
So here's what you need to consider on extraction, to prevent the pipelines from breaking and to keept them running smoothly:

  1. **Hardware limits:** Be mindful of memory (RAM) and storage (disk space). Overloading these can crash your system.
  2. **Network reliability:** Networks can fail! Always account for retries to make your pipelines more robust.
        * Tip: Use libraries like dlt that have built-in retry mechanisms.
  3. **API rate limits:** APIs often restrict the number of requests you can make in a given time.
        * Tip: Check the API documentation to understand its limits (e.g. Zendesk, Shopify).

There are even more challenges to consider when working with APIs - such as **pagination and authentication**. Let's explore how to handle these effectively when working with **REST APIs.**

### Common Challenges: API Interaction Challenges

1. **Authentication**
  - API keys
  - OAuth Tokens
  - Basic Authentication

2. **Memory Management**
  - Limited Memory
  - Streaming

3. **Pagination**
  - Pages
  - Data Chunks

4. **Rate limits**
  - Pause requests
  - Retry-After Header

APIs:

* Monitor Rate Limits - Check API headers for remaining requests.
* Pause Requests - Delay further requests if limits are near.
* Implement Retries - Retry failed requests after a delay.

#### 1. API Rate Limit

In [12]:
import time

url = 'https://api.github.com/rate_limit'  # some webs have a special "end point" to see if we reached the limit of requests

remaining = requests.get(url).json()['rate']['remaining']

if remaining == 0:
    time.sleep(30)

remaining

59

#### 2. Authentication

Many APIs require an **API key or token** to access data securely. Without authentication, requests may be limited or denied.

**Types of Authentication in APIs:**

  - **API Keys** - A simple token included in the request header or URL.
  - **OAuth Tokens** - A more secure authentication method requiring user authorization.
  - **Basic Authentication** - Using a username and password (less common today).

Never share your API token publicly! Store it in environment variables or use a secure secrets manager.


Example:
    In this example, we'll request data from Github API.

In [13]:
url = 'https://api.github.com/user'

requests.get(url).json()

{'message': 'Requires authentication',
 'documentation_url': 'https://docs.github.com/rest',
 'status': '401'}

In [2]:
# If we were in google colab

'''from google.colab import userdata
API_TOKEN = userdata.get('ACCESS_TOKEN')

headers = {
            'Authorization': f'Bearer {API_TOKEN}'
}
'''
'''
ACCESS_TOKEN = '-' # I generated this on GitHub

headers = {
            'Authorization': f'Bearer {ACCESS_TOKEN}'
}

url = 'https://api.github.com/user'

requests.get(url, headers=headers).json()

'''

"\nACCESS_TOKEN = '-' # I generated this on GitHub\n\nheaders = {\n            'Authorization': f'Bearer {ACCESS_TOKEN}'\n}\n\nurl = 'https://api.github.com/user'\n\nrequests.get(url, headers=headers).json()\n\n"

#### 3. Pagination

Many APIs return data in **chunks (or pages)** rather than sending everything at once. This prevents **overloading the server** and improves performance, especially for large datasets. To retrieve **all the data**, we need to make multiple requests and keep track of pages until we reach the last one.

**API Data Retrieval Process**

Initiate Data Request >>> Check for More Data >>> Repeat Process


- **Pagination:** When there's no more data, the API returns no links to the next page.
- **Details:**

    * **Method:** GET
    * **URL:** https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/events
    * **Parameters:**
        - page: Integers (page number), defaults to 1.
        - per_page: The number of results per page (max 100). Defaults to 30.

  When the response is paginated, the response headers will include a link header. If the endpoint does not support pagination. or if all results fit on a single page, the link header will be omitted.

  The link header contains URLs that you can use to fetch additional pages of results. For example, the previous, next, first, and last page of results.

This script demonstrates how to handle paginated responses by automatically requesting the next page of data until all pages are retrieved:

In [16]:
url = 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/events?page=9'

response = requests.get(url)

response

<Response [200]>

In [17]:
response.headers['Link']

'<https://api.github.com/repositories/419661684/events?page=8>; rel="prev", <https://api.github.com/repositories/419661684/events?page=10>; rel="next", <https://api.github.com/repositories/419661684/events?page=10>; rel="last", <https://api.github.com/repositories/419661684/events?page=1>; rel="first"'

In [18]:
response.links # Because we don't want to parse

{'prev': {'url': 'https://api.github.com/repositories/419661684/events?page=8',
  'rel': 'prev'},
 'next': {'url': 'https://api.github.com/repositories/419661684/events?page=10',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/419661684/events?page=10',
  'rel': 'last'},
 'first': {'url': 'https://api.github.com/repositories/419661684/events?page=1',
  'rel': 'first'}}

In [19]:
response.links['next']['url'] # If there is no next, i might throw an error

'https://api.github.com/repositories/419661684/events?page=10'

In [20]:
url = 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/events' # ?page=9

while True:
    response = requests.get(url)
    data = response.json()
    print(len(data))
    
    if 'next' not in response.links:
        break
    url = response.links['next']['url']

30
30
30
30
30
30
30
30
29
30


**What happens here:**
   - It starts at **page 1** and makes a **GET request** to the API.
   - It retrieves **JSON data.**
   - It looks for the **"next" page URL** in the response headers.
   - If a **next page exists**, updates BASE_URL and requests more data.
   - If there's **no next page**, stops fetching and ends the loop.


Different APIs handle pagination differently (some use offsets, cursors, page numbers, or tokens instead of links). Always check the API documentation for the correct method!

#### 4. Avoiding memory issues during extraction

To prevent your pipeline from crashing, you need to control memory usage.

**Challenges with memory**
  * Many pipelines run on systems with limited memory, like serverless functions or shared clusters.
  * If you try to load all the data into memory at once, it can crash the entire system.
  * Even disk space can become an issue if you're storing large amounts of data.


Memory Limitations : Pipelines may crash if data is loaded all at once.

Disk Space Issues : Storing large amounts of data can lead to problems.

**The solution: batch processing/ streeaming data**

**Streaming** means processing data in small chunks or events, rather than loading everything at once. This keeps memory usage low and ensures your pipeline remains efficient.

As a data engineer, you'll use streaming to transfer data between buffers, such as:

  * from APIs to local files.
  * from Webhooks to event queues.
  * from Event queues (like Kafka) to storage buckets.


In [35]:
def events_getter():
    url = 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp/events'

    while True:
        response = requests.get(url)
        data = response.json()
        yield data
        
        if 'next' not in response.links:
            break
        url = response.links['next']['url']

In [36]:
events_pages = events_getter()

for events_page in events_pages:
    print(events_page)

[{'id': '5415617638', 'type': 'WatchEvent', 'actor': {'id': 98461747, 'login': 'Heinmyatko1996', 'display_login': 'Heinmyatko1996', 'gravatar_id': '', 'url': 'https://api.github.com/users/Heinmyatko1996', 'avatar_url': 'https://avatars.githubusercontent.com/u/98461747?'}, 'repo': {'id': 419661684, 'name': 'DataTalksClub/data-engineering-zoomcamp', 'url': 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp'}, 'payload': {'action': 'started'}, 'public': True, 'created_at': '2025-12-19T16:44:17Z', 'org': {'id': 72699292, 'login': 'DataTalksClub', 'gravatar_id': '', 'url': 'https://api.github.com/orgs/DataTalksClub', 'avatar_url': 'https://avatars.githubusercontent.com/u/72699292?'}}, {'id': '5415481638', 'type': 'ForkEvent', 'actor': {'id': 67496079, 'login': 'tessadt', 'display_login': 'tessadt', 'gravatar_id': '', 'url': 'https://api.github.com/users/tessadt', 'avatar_url': 'https://avatars.githubusercontent.com/u/67496079?'}, 'repo': {'id': 419661684, 'name': 'DataTal

The snipped below demonstrates streaming (chunked) data processing, where data is processed in small batches instead of loading everything into memory at once. This method helps optimize performance and reduce memory usage.

In this approach to grabbing data from APIs, there are both pros and cons:

✅Pros: **Easy memory management** since the API returns data in small pages or events.

❌Cons: **Low throughput** because data transfer is limited by API constraints (rate limits, response time).

To simplify data extraction, use specialized tools that follow best practices like streaming - for example, dlt (data load tool). It efficiently processes data while **keeping memory usage low** and **leveraging parallelism** for better performance.

Well, you've successfully **extracted** tha data - great! But raw data isn't always ready to use. Now, you need to **process**, **clean** and **structure** it before it can be loaded into a data lake or data warehouse.

## Normalizing data

You often hear that data professionals spend most of their time **"cleaning" data** - but what does that actually mean?

Data cleaning typically involves two key steps:

 1. **Normalizing data** - Structuring and standardizing data **without changing its meaning.**
 2. **Filtering data for a specific use case** - Selecting or modifying data **in a way that changes its meaning** to fit the analysis.


 ### Data cleaning: more than just fixing errors

 A big part of **data cleaning** is actually **metadata work** - ensuring data is structured and standardized so it can be used effectively.

 **Metadata tasks in data cleaning:**

✅ **Add types** - Convert strings to numbers, timestamps, etc.

✅ **Rename columns** - Ensure names follow a standard format (e.g. no special characters).

✅ **Flatten nested dictionaries** - Bring values from nested dictionaries into the top-level row. Simplify data structure.

✅ **Unnest lists/arrays** - Convert lists into **child tables** since they can't be stored directly in a flat format.

**We'll look at a practical example next, as these concepts are easier to understand with real data**


**Why prepare data? Why not use JSON directly?**

While JSON is great format for **data transfer**, it's not ideal for analysis.

- **No enforced schema** - We don't always know what fields exist in a JSON document.
- **Inconsistent data types** - A field like age might appear as 25, "twenty five", or 25.00, which can break downstream applications. 
- **Hard to process** - if we need to group data by day, we must manually convert date strings to timestamps.
- **Memory-heavy** - JSON requires reading the entire file into memory, unlike databases or columnar formats that allow scanning just the necessary fields.
- **Slow for aggregation and search** - JSON is not optimized for quick lookups or aggregations like columnar formats (e.g. Parquet).

JSON is great for **data exchange** but **not for direct analytical use**. To make data useful, we need to **normalize it** - flattening, typing, and structuring it for efficiency.

#### Normalization Example

To understand what we're working with, let's look at a sample record from the API:

In [38]:
event = events_page[0]
event

{'id': '5172165152',
 'type': 'WatchEvent',
 'actor': {'id': 42970073,
  'login': 'Blake-Loveland',
  'display_login': 'Blake-Loveland',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Blake-Loveland',
  'avatar_url': 'https://avatars.githubusercontent.com/u/42970073?'},
 'repo': {'id': 419661684,
  'name': 'DataTalksClub/data-engineering-zoomcamp',
  'url': 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp'},
 'payload': {'action': 'started'},
 'public': True,
 'created_at': '2025-12-08T20:19:37Z',
 'org': {'id': 72699292,
  'login': 'DataTalksClub',
  'gravatar_id': '',
  'url': 'https://api.github.com/orgs/DataTalksClub',
  'avatar_url': 'https://avatars.githubusercontent.com/u/72699292?'}}

The data we retrieved from the API has **a nested JSON format.** Let's unnest it!

 It means that any **nested structures** (like dictionaries and lists) have to be flattened, to make it easier to store and query in a database or a dataframe.

 #### How to process this data?

 1. **Flatten nested fields:**

 - For example, fields actor, repo, payload and org are nested and we should extract all the necessary data:

 'actor': {

    'id': 198386041,

    'login': 'Anqi0607',

    ...
 }

 to

 'actor__id': 198386041,

 'actor__login': 'Anqi0607',
 
 ...

In [39]:
event

{'id': '5172165152',
 'type': 'WatchEvent',
 'actor': {'id': 42970073,
  'login': 'Blake-Loveland',
  'display_login': 'Blake-Loveland',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Blake-Loveland',
  'avatar_url': 'https://avatars.githubusercontent.com/u/42970073?'},
 'repo': {'id': 419661684,
  'name': 'DataTalksClub/data-engineering-zoomcamp',
  'url': 'https://api.github.com/repos/DataTalksClub/data-engineering-zoomcamp'},
 'payload': {'action': 'started'},
 'public': True,
 'created_at': '2025-12-08T20:19:37Z',
 'org': {'id': 72699292,
  'login': 'DataTalksClub',
  'gravatar_id': '',
  'url': 'https://api.github.com/orgs/DataTalksClub',
  'avatar_url': 'https://avatars.githubusercontent.com/u/72699292?'}}

In [40]:
def process_event(event):
    result = {}

    result['id'] = event['id']
    result['type'] = event['type']
    result['public'] = event['public']
    result['created_at'] = event['created_at'] 

    result['actor__id'] = event['actor']['id']
    result['actor__login'] = event['actor']['login']

    return result

In [41]:
# process_event(event)

processed_events = []

for event in events_page:
    processed_event = process_event(event)
    processed_events.append(processed_event)

processed_events

[{'id': '5172165152',
  'type': 'WatchEvent',
  'public': True,
  'created_at': '2025-12-08T20:19:37Z',
  'actor__id': 42970073,
  'actor__login': 'Blake-Loveland'},
 {'id': '5171661461',
  'type': 'WatchEvent',
  'public': True,
  'created_at': '2025-12-08T19:52:13Z',
  'actor__id': 248546385,
  'actor__login': 'dsu-mn'},
 {'id': '5168709509',
  'type': 'WatchEvent',
  'public': True,
  'created_at': '2025-12-08T17:25:48Z',
  'actor__id': 124050566,
  'actor__login': 'wygol'},
 {'id': '5168157790',
  'type': 'WatchEvent',
  'public': True,
  'created_at': '2025-12-08T17:01:32Z',
  'actor__id': 248033104,
  'actor__login': 'llmaskcell-afk'},
 {'id': '5167957314',
  'type': 'WatchEvent',
  'public': True,
  'created_at': '2025-12-08T16:52:54Z',
  'actor__id': 121841784,
  'actor__login': 'i-wijaya-agriaku'},
 {'id': '5165170769',
  'type': 'ForkEvent',
  'public': True,
  'created_at': '2025-12-08T15:05:46Z',
  'actor__id': 54820671,
  'actor__login': 'sarkriti'},
 {'id': '5165167401',


2. **Convert timestamps:**

- Originally, timestamps might have been stored as ISO datetime strings:

"created_at": "2024-06-12T14:28:46Z"

You can store them as they are, but in some cases, you may need to **convert** them to timestamps.

- Now, they are **formatted as Unix timestamp:**

"created_at": "1718202526"

In [42]:
from datetime import datetime
# datetime.fromisoformat timestamp()

def process_event(event):
    result = {}

    result['id'] = event['id']
    result['type'] = event['type']
    result['public'] = event['public']
    
    
    parsed_timestamp = datetime.fromisoformat(event['created_at'])
    result['created_at'] = parsed_timestamp.timestamp()
    result['actor__id'] = event['actor']['id']
    result['actor__login'] = event['actor']['login']

    return result

process_event(event)

{'id': '5131912045',
 'type': 'WatchEvent',
 'public': True,
 'created_at': 1764993148.0,
 'actor__id': 56884740,
 'actor__login': 'namphuog'}

In [43]:
all_data = []

pages = events_getter()

for page in pages:
    all_data.extend(page)

len(all_data)

299

3. **Unnest lists:**

- The original structure might include a nested list:

'is_template': False,

'web_commit_signoff_required': False,

'topics': ['data-engineering', 'dbt', 'docker', 'kafka', 'kestra', 'spark'],

'visibility': 'public',

'forks': 6093,

* Since lists **cannot be stored directly in a database table**, they were likely **moved to a separate table**. 

In [44]:
def process_event(event):
    result = {}

    result['id'] = event['id']
    result['type'] = event['type']
    result['public'] = event['public']

    parsed_timestamp = datetime.fromisoformat(event['created_at'])
    result['created_at'] = parsed_timestamp.timestamp()

    result['actor__id'] = event['actor']['id']
    result['actor__login'] = event['actor']['login']

    topics = event.get('payload', {}).get('pull_request', {}).get('base', {}).get('repo', {}).get('topics', [])

    processed_topics = []
    for topic in topics:
        processed_topics = {
            'event_id': event['id'],
            'topic': topic
        }
        processed_topics.append(processed_topics)

    return result, processed_topics

In [45]:
processed_events = []
processed_topics = []

for event in all_data:
    processed_event, topics = process_event(event)
    processed_events.append(processed_event)
    processed_topics.extend(topics)

print(processed_events[:5])
print(processed_topics[:5])

[{'id': '5415617638', 'type': 'WatchEvent', 'public': True, 'created_at': 1766162657.0, 'actor__id': 98461747, 'actor__login': 'Heinmyatko1996'}, {'id': '5415481638', 'type': 'ForkEvent', 'public': True, 'created_at': 1766162242.0, 'actor__id': 67496079, 'actor__login': 'tessadt'}, {'id': '5414379751', 'type': 'WatchEvent', 'public': True, 'created_at': 1766158831.0, 'actor__id': 13095925, 'actor__login': 'piyush-24sharma'}, {'id': '5411726096', 'type': 'WatchEvent', 'public': True, 'created_at': 1766150733.0, 'actor__id': 1312845, 'actor__login': 'qivhou'}, {'id': '5410423440', 'type': 'WatchEvent', 'public': True, 'created_at': 1766146431.0, 'actor__id': 130214920, 'actor__login': 's0nnyc'}]
[]


**Real-world data is rarely clean!** We often receive raw, nested, and inconsistent data. This is why the **normalization process** is so important. It **prepares** the data for efficient storage and analysis. dlt (data load tool) simplifies the **normalization process**, automatically transforming raw data into a **structured, clean format** that is ready for storage and analysis.

### Loading data

Now that we've covered **extracting** and **normalizing** data, the final step is **loading** the data **into a destination**. This is where the processed data is stored, making it ready for querying, analysis, or further transformations.

##### How data loading usually happens

Before dlt, data engineers had to manually handle **schema validation, batch processing, error handling, and retries** for every destination. This process becomes especially complex when loading data into **data warehouses and data lakes**, where performance optimization, partitioning and incremental updates are critical.

**Example: Loading data into database**

Here's and DuckDB script that creates two separate tables:

1. Watch_events --> Stores WatchEvents data
2. pull_request_events --> Stores PullRequestEvent data

A basic pipeline requires:
  1. Setting up a database connection.
  2. Creating tables and defining schemas.
  3. Writing queries to insert/update data.
  4. Handling schema changes manually. 

In [47]:
#%pip install duckdb

In [48]:
import duckdb

# 1. Create a connection to a DuckDB database

conn = duckdb.connect('github_events.duckdb')

In [50]:
processed_events[0]

{'id': '5415617638',
 'type': 'WatchEvent',
 'public': True,
 'created_at': 1766162657.0,
 'actor__id': 98461747,
 'actor__login': 'Heinmyatko1996'}

In [49]:
# 2. Create the `github_events` table

conn.execute("""
CREATE TABLE IF NOT EXISTS github_events (
             id TEXT PRIMARY KEY,
             type TEXT,
             public BOOLEAN,
             created_at DOUBLE,
             actor__id BIGINT,
             actor__login TEXT
);
""")

In [51]:
flattened_data = [
    (
        record['id'],
        record['type'],
        record['public'],
        record['created_at'],
        record['actor__id'],
        record['actor__login']

    )
    for record in processed_events
]

# 3. Insert data into the `github_events` table

conn.executemany("""
INSERT INTO github_events (id, type, public, created_at, actor__id, actor__login)
VALUES (?, ?, ?, ?, ?, ?)
ON CONFLICT (id) DO NOTHING;
""", flattened_data)

In [52]:
df = conn.execute("SELECT * FROM github_events LIMIT 5").df()
df.head()

,id,type,public,created_at,actor__id,actor__login
0,5415617638,WatchEvent,True,1.766163e+09,98461747,Heinmyatko1996
1,5415481638,ForkEvent,True,1.766162e+09,67496079,tessadt
2,5414379751,WatchEvent,True,1.766159e+09,13095925,piyush-24sharma
3,5411726096,WatchEvent,True,1.766151e+09,1312845,qivhou
4,5410423440,WatchEvent,True,1.766146e+09,130214920,s0nnyc


In [53]:
conn.close()

Problems:
* **Schema management is manual** - If the schema changes, you need to update table structures manually.
* **No automatic retries** - If the network falls, data may be lost.
* **No incremental loading** - Every run reloads everything, making it slow and expensive.
* **More code to maintain** - A simple pipeline quickly becomes complex.

## **Dynamic schema handling in DuckDB**

Many engineers take shortcuts when loading data, thinking: it works, so why overcomplicate it?

But this mindset leads to technical debt and painful maintenance cycles:

 * **You will be stuck doing maintenance** - Quick scripts break as requirements evolve, turning you into a permanent firefighter.
 * **Fixing ingestion in production is 10x harder** - Once data is in use, breaking changes mean lost trust and late-night emergencies.
 * **Bad data loading is like bad architecture** - Just as a poor data model causes downstream issues, unreliable ingestion causes instability across the entire stack.

 In real-world data engineering, incoming data estructures often change - new columns appear, and old ones may disappear. Instead of manually updating schemas, we can automate schema evolution to ensure our pipelines keep running smoothly. 

In [54]:
def process_event(event):
    result = {}

    result['id'] = event['id']
    result['type'] = event['type']
    result['public'] = event['public']

    parsed_timestamp = datetime.fromisoformat(event['created_at'])
    result['created_at'] = parsed_timestamp.timestamp()

    result['actor__id'] = event['actor']['id']
    result['actor__login'] = event['actor']['login']

    result['repo__id'] = event['repo']['id']


    topics = event.get('payload', {}).get('pull_request', {}).get('base', {}).get('repo', {}).get('topics', [])

    processed_topics = []
    for topic in topics:   
        processed_topic = {
            'event_id': event['id'],
            'topic': topic
        }
        processed_topics.append(processed_topic)
    
    return result, processed_topics

In [55]:
processed_events = []
processed_topics = []

for event in all_data:
    processed_event, topics = process_event(event)
    processed_events.append(processed_event)
    processed_topics.extend(topics)

print(processed_events[:5])
print(processed_topics[:5])

[{'id': '5415617638', 'type': 'WatchEvent', 'public': True, 'created_at': 1766162657.0, 'actor__id': 98461747, 'actor__login': 'Heinmyatko1996', 'repo__id': 419661684}, {'id': '5415481638', 'type': 'ForkEvent', 'public': True, 'created_at': 1766162242.0, 'actor__id': 67496079, 'actor__login': 'tessadt', 'repo__id': 419661684}, {'id': '5414379751', 'type': 'WatchEvent', 'public': True, 'created_at': 1766158831.0, 'actor__id': 13095925, 'actor__login': 'piyush-24sharma', 'repo__id': 419661684}, {'id': '5411726096', 'type': 'WatchEvent', 'public': True, 'created_at': 1766150733.0, 'actor__id': 1312845, 'actor__login': 'qivhou', 'repo__id': 419661684}, {'id': '5410423440', 'type': 'WatchEvent', 'public': True, 'created_at': 1766146431.0, 'actor__id': 130214920, 'actor__login': 's0nnyc', 'repo__id': 419661684}]
[]


In [56]:
# 1. Create a connection to a DuckDB database

conn = duckdb.connect('github_events.duckdb')

In [57]:
current_columns = {row[1] for row in conn.execute("PRAGMA table_info('github_events');").fetchall()}
print(current_columns)

{'actor__login', 'type', 'id', 'public', 'actor__id', 'created_at'}


In [58]:
# 3. Detect and add new columns dynamically

for record in processed_events[10:]:
    for key in record.keys():
        if key not in current_columns:
            col_type = 'TEXT'  # Default data type
            if isinstance(record[key], bool):
                col_type = 'BOOLEAN'
            elif isinstance(record[key], int):
                col_type = 'BIGINT'
            elif isinstance(record[key], float):
                col_type = 'DOUBLE'
            print(f"ALTER TABLE github_events ADD COLUMN {key} {col_type};")
            alter_query = f"ALTER TABLE github_events ADD COLUMN {key} {col_type};"
            conn.execute(alter_query)
            print(f"Added new column: {key} ({col_type})")
            current_columns.add(key) # Update schema tracking

ALTER TABLE github_events ADD COLUMN repo__id BIGINT;
Added new column: repo__id (BIGINT)


In [59]:
# 4. Prepare data for insertion (handle missing fields)
columns = sorted(current_columns)    # Maintain consistent column order
flattened_data = [
    tuple(record.get(col, None) for col in columns) # Fill missing values with NULL
    for record in processed_events
]

# 5. Construct dynamic SQL for insertion
placeholders = ', '.join(["?" for _ in columns])
columns_str = ', '.join(columns)

insert_query = f"""
INSERT INTO github_events ({columns_str})
VALUES ({placeholders})
ON CONFLICT (id) DO UPDATE SET {', '.join([f"{col}=excluded.{col}" for col in columns if col != 'id'])};
"""

In [60]:
# 6. Insert data into DuckDB
conn.executemany(insert_query, flattened_data)

In [61]:
# 7. Query the table
df = conn.execute("""SELECT * FROM github_events""").df() #  LIMIT 5

df.head()

,id,type,public,created_at,actor__id,actor__login,repo__id
0,5415617638,WatchEvent,True,1.766163e+09,98461747,Heinmyatko1996,419661684
1,5415481638,ForkEvent,True,1.766162e+09,67496079,tessadt,419661684
2,5414379751,WatchEvent,True,1.766159e+09,13095925,piyush-24sharma,419661684
3,5411726096,WatchEvent,True,1.766151e+09,1312845,qivhou,419661684
4,5410423440,WatchEvent,True,1.766146e+09,130214920,s0nnyc,419661684


In [62]:
# conn.close()

**What's happening here?**

Step                    Action
1. Connects to DuckDB
2. Fetches current schema from DuckDB
3. Detects and **adds new columns dynamically**
4. Fills missing values with NULL to ensure smooth inserts.
5. Dynamically constructs an **INSERT or UPDATE (UPSERT)** query.
6. Inserts new data while updating existing records.
7. Queries the data.

## Welcome to real-world Data Engineering

This example shows how to **dynamically handle schema evolution** in DuckDB. But this is just the beginning!

### **What else can you do?**

 * **Dynamic table creation** - Automatically generate tables for new datasets.
 * **Detect data type changes** - Ensure schema consistency as data evolves.
 * **Enable incremental loading** - Efficiently update data instead of full reloads.
 * **Optimize performance** - Use **parallelization** for faster ingestion.
 * **Ensure atomicity** - Make updates fail-safe with **transactions**.


### ⚠️**The reality: it's a Full-Time job!**

Imagine how your **codebase will grow** as you extend this logic for productions destinations (PostgreSQL, BigQuery, etc).

**Production challenges:**

❌**Manual work & boilerplate code** - Schema tracking, type conversion, migrations...

❌**Performance tuning** - Parallel execution, indexing, partitioning...

❌**Testing & validation** - Unit tests, integration tests, rollback strategies...

❌**Scaling gets messy** - What works for 1M rows breaks at 100M.

❌**Maintaining & keeping it up-to-date** - Because data never stops changing.


### 💡**Skip all these struggles: use dlt**

Instead of **reinventing the wheel**, you can **skip all this manual work** and just use **dlt**:
✅**Automatic schema management** - Handles schema changes for you. New columns, type changes? Handled.

✅**Incremental loading** - Ingest only new or updated records efficiently.

✅**Works with your stack** - PostgreSQL, BigQuery, Snowflake, DuckDB, and more.

✅**Built-in parallelization** - Faster ingestion with zero manual tuning.

✅**Automatic & reliable** - Ensures data consistency with automatic retries.

This is the **real work of a data platform engineer** - making pipelines scalable, robust, and maintainable. But with dlt, you get all of this **out of the box**.


So, what's your next step? Schema versioning? Building data lakes? Automatic retry mechanisms?

The deeper you go, the more you realize: building a production-grade data pipeline is not just a script - **it's an entire platform.**

🔥**Or you can just use dlt and focus on what really matters - your data.** 🔥


# Part two: 
# Data engineering for senior data professionals

In [1]:
#%pip install dlt[duckdb] # install dlt with all the necessary DuckDB dependencies